<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a>, ©<a href="https://apps.ep.jhu.edu/course-homepages/3765-605-742-deep-neural-networks" target="_blank">JHU</a> 2021 onwards</font></small><hr style="margin:0;background-color:silver">

# **[2🏆🌌Stellar](https://www.kaggle.com/competitions/30jan23JH-stellar/rules)**

See [**instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running and naming your Colab notebooks.



**(Optioal) CONSENT.** If ok with sharing your Colab for educational purposes, please check the box below with "X".

<mark>[ .X. ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purpose. We understand that sharing is optional and this decision will not affect our grade in any way. 

In [1]:
# from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

In [2]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 30jan23cml-stellar        # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
lb =!kaggle competitions leaderboard --show                   # print public leaderboard

- competition is now set to: 30jan23cml-stellar


In [3]:
# lb0 = [(int(l[:7]), l[9:60].strip(), l[60:80].strip(), float(l[81:].strip())) for l in lb[3:]]
# dflb = pd.DataFrame(lb0, columns=['id','team','dt','score'])
# dflb = dflb[dflb.team.str.contains('🐯') | dflb.team.str.lower().str.contains('baseline')]
# dflb.sort_values('dt', ascending=False)

In [4]:
%%time
%%capture
%reset -f
# import pandas as pd
# import sklearn
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.preprocessing import LabelEncoder
# from sklearn.neighbors import LocalOutlierFactor
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all" 
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LogisticRegression as LR
# from sklearn.model_selection import train_test_split
# from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA, LinearDiscriminantAnalysis as LDA
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('max_columns', 20, 'precision', 2, 'display.max_rows', 4)

CPU times: user 1.36 s, sys: 152 ms, total: 1.51 s
Wall time: 2.36 s


In [5]:
# df.info()   # observe datatypes and any missing values

In [6]:
# Change string labels to numbers in order of increasing size of the entity (Star < Quasi Star < Galaxy)
# df.Class = df.Class.apply(lambda C: -1 if C=='S' else 0 if C=='Q' else 1 if C=='G' else None) 

In [7]:
# vX = df.query('Class!=Class').drop('Class', axis=1)  # slice a test sample
# tXY = df.query('Class==Class')                       # slice training sample
# tX, tY = tXY.drop('Class', axis=1), tXY.Class        # split into training I/O

In [8]:
# def ScatterCorrHist(df):
#   def corrdot(*args, **kwargs):
#     # credit: https://stackoverflow.com/questions/48139899
#     corr_r = args[0].corr(args[1], 'pearson')
#     corr_text = f"{corr_r:2.2f}".replace("0.", ".")
#     ax = plt.gca();
#     ax.set_axis_off();
#     msz = abs(corr_r) * 5000   # marker size
#     fsz = abs(corr_r) * 40 + 5 # font size
#     ax.scatter([.5], [.5], msz, [corr_r], alpha=0.5, cmap='coolwarm', vmin=-1, vmax=1, transform=ax.transAxes)
#     ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction", ha='center', va='center', fontsize=fsz)

#   sns.set(style='white', font_scale=.8);
#   g = sns.PairGrid(df, aspect=1, diag_sharey=False);
#   g.fig.set_size_inches(20,10)
#   g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color':'red'}, scatter_kws={'s':1});
#   g.map_diag(sns.histplot, kde_kws={'color':'black'});
#   g.map_upper(corrdot);
#   g.fig.suptitle("Scatter plot, Correlations and histograms on diagonal", y=1);
#   _ = plt.subplots_adjust(hspace=0.02, wspace=0.02);
#   _ = plt.show();

# # ScatterCorrHist(tXY.head(200))

In [9]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**
 
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc. 
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

In [10]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [11]:
df = pd.read_csv('XY_Stellar.csv')

In [12]:
testing = df[df['Class'] != df['Class']]
testing = testing.drop('Class', axis=1)

In [13]:
df = df.dropna()

In [14]:
enc1 = LabelEncoder()
enc1.fit(df['Class'])
df['Class'] = enc1.transform(df['Class'])

LabelEncoder()

Удалим бесполезные признаки, не дающие никакой информации для предсказания класса.

In [15]:
testing = testing.drop(['alpha','delta','run_ID','cam_col','field_ID','fiber_ID'], axis = 1)

In [16]:
df = df.drop(['alpha','delta','run_ID','cam_col','field_ID','fiber_ID'], axis = 1)

Почистим датасет от выбросов, влияющих на качество обучения. 

In [17]:
df = df.drop(df[(df['u'] < 0) | (df['u'] > 50)].index)
df = df.drop(df[(df['r'] < 0) | (df['r'] > 40)].index)
df = df.drop(df[(df['g'] < 0) | (df['g'] > 40)].index)
df = df.drop(df[(df['i'] < 0) | (df['i'] > 40)].index)
df = df.drop(df[(df['z'] < 0) | (df['z'] > 40)].index)

Удалим еще выбросы с помощью выявителя Outliers.

In [18]:
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor()
y_pred = clf.fit_predict(df) 

In [19]:
x_score = clf.negative_outlier_factor_
outlier_score = pd.DataFrame(x_score, index=df.index)

In [20]:
df = df.drop(outlier_score[outlier_score[0] < -2].index)

In [21]:
df_y = df['Class']
df_x = df.drop('Class', axis=1)

Сделаем препроцессинг и заскейлим датасет.

In [22]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_x)
df_x = scaler.transform(df_x)

StandardScaler()

Создадим полиномиальные признаки 2 степени.

In [23]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2).fit(df_x)
df_x = poly.transform(df_x)

Снова заскейлим данные.

In [24]:
scaler_after = StandardScaler()
scaler_after.fit(df_x)
df_x = scaler_after.transform(df_x)

StandardScaler()

In [25]:
testing = scaler.transform(testing)
testing = poly.transform(testing)
testing = scaler_after.transform(testing)

**Student's answer:**

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful. 

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

Below is a baseline model that produces the result on Kaggle leaderboard (LB).

Мы попробовали все доступные модели в данном соревновании, но лучший скор по дефолту показал классический логрег, поэтому используем его.

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.2, random_state = 99)

In [27]:
dl = len(x_train) // 2
x_train_first = x_train[:dl,]
x_train_second = x_train[dl:,]
y_train_first = y_train[:dl,]
y_train_second = y_train[dl:,]

Возьмем классификатор OneVSRest и обучим его на половине данных.

In [28]:
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.linear_model import LogisticRegression

orc = OneVsRestClassifier(LogisticRegression(solver='lbfgs', max_iter=400, penalty='none', warm_start=True)).fit(x_train_first, y_train_first)

sklearn.metrics.accuracy_score(y_test, orc.predict(x_test))

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

0.9662079558634525

In [29]:
x_train_second = pd.DataFrame(x_train_second)
x_train_second[45] = orc.predict(x_train_second)


In [30]:
x_test_new = pd.DataFrame(x_test).copy()
x_test_new[45] = orc.predict(x_test)


Далее предскажем 1 классификатором класс на 2 половине и подадим это в обучающую выборку 2 классификатора OneVsOne.

In [31]:
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
ooc = OneVsOneClassifier(
    LogisticRegression(solver='lbfgs',
                         penalty='none',
                         max_iter=1000, #1200
                         tol = 1e-4,
                         random_state=69)).fit(x_train_second, y_train_second)

sklearn.metrics.accuracy_score(y_test, ooc.predict(x_test_new))
#0.9702830632268581

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9704397981254506

Сделаем предикт.

In [32]:
testing = pd.DataFrame(testing)


In [33]:
testing[45] = orc.predict(testing)
res = ooc.predict(testing)
# pd.DataFrame(res)

In [34]:
ToCSV = lambda df, fname: df.to_csv(f'{fname}.csv', index_label='id')
submit_final = pd.DataFrame(enc1.inverse_transform(ooc.predict(testing)), columns=['Class'])
submit_final.index += 1
ToCSV(submit_final, 'orc_ooc_1')

**Student's answer:**

# **References:**

1. Remember to cite your sources here as well! At the least, your textbook should be cited. Google Scholar allows you to effortlessly copy/paste an APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources to help your peers learn from these (and to avoid plagiarism claims).

Ссылки на источники, в которых есть информация по датасету:

https://en.wikipedia.org/wiki/Quasi-star

https://en.wikipedia.org/wiki/Star

https://en.wikipedia.org/wiki/Galaxy

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [35]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 55 sec


## 💡**Starter Ideas**

1. Tune model hyperparameters
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest missclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.
1. Do scatter plots show piecewise linear shape? Can a separate linear model be used on each support, or can the pattern be linearized via transformations?
1. How are date/categorical features treated by the model? Is there a [better way](https://www.google.com/search?q=ways+to+encode+categorical+data) to encode these (perhaps, ordinal) features? 
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?
1. Learn astronomy domain and features: [🎦](https://www.youtube.com/results?search_query=Quasi-star), [quasi-star](https://en.wikipedia.org/wiki/Quasi-star), [star](https://en.wikipedia.org/wiki/Star), [galaxy](https://en.wikipedia.org/wiki/Galaxy), [📃](https://arxiv.org/abs/2112.02026)


Team 5 - Shamteev Georgy, Andrew Vasin, Maksim Kislov